In [ ]:
import requests as re
import foursquare
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library

In [ ]:
#Open dataFrame with neighbourhoods
df=pd.read_csv('nbhoods_new.csv')
df.head(5)

In [ ]:
#Check neighborhoods with similar names to make sure they are correct
df[df['Neighborhood'].str.contains('Chelsea')]

In [ ]:
#Check what kind of Bbox we have
print(df['Bbox'][4])#Seemd like we cannot use it as it has repeating pairs of coordinates

In [ ]:
#Check names of all boroughs
df['Borough'].unique()

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

In [ ]:
#Plot the neighborhoods
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map - IS THAT POSSIBLE TO DO AREAS INSTEAD OF POINTS?
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [ ]:
#Set up Foursquare - paste credentials
CLIENT_ID = FOURSQUARE_CLIENT_ID 
CLIENT_SECRET = FOURSQUARE_CLIENT_SECRET
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
#Get data for MANHATTAN
manhattan_data=df.loc[df['Borough']=='Manhattan'].copy()

In [ ]:
manhattan_data

In [ ]:
manhattan_data=manhattan_data.reset_index(drop=True)

In [ ]:
manhattan_data.loc[0, 'Latitude'] 

In [ ]:
#Define variables for the function
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
#Define parameters for the function
radius = 908 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

In [ ]:
results = requests.get(url=url).json()
results

In [ ]:
#def getNearbyVenues(names, latitudes, longitudes, radius=500):
    '''
    Returns the dataframe with neighborhood, venue name, venue lat, venue lon, venue category.

            Parameters:
                    names (list): string
                    latitudes (list): float
                    longitudes (list): float

            Returns:
                    nearby_venues (dataFrame): dataFrame
    '''

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#       print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
#           lat, 
#           lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
#                 'Neighborhood Latitude', 
#                 'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#Get Manhattan data
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

In [ ]:
print(manhattan_venues.shape)
manhattan_venues.head()

In [ ]:
#Get data for BRONX
bronx_data=df.loc[df['Borough']=='Bronx'].copy()

In [ ]:
bronx_data

In [ ]:
bronx_data=bronx_data.reset_index(drop=True)

In [ ]:
bronx_data.loc[0, 'Latitude'] 

In [ ]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

In [ ]:
print(bronx_venues.shape)
bronx_venues.head()

In [ ]:
#Get venues for Brooklyn
brooklyn_data=df.loc[df['Borough']=='Brooklyn'].copy()

In [ ]:
brooklyn_data=brooklyn_data.reset_index(drop=True)

In [ ]:
brooklyn_data.loc[0, 'Latitude'] 

In [ ]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

In [ ]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

In [ ]:
#Get venues for Queens
queens_data=df.loc[df['Borough']=='Queens'].copy()

In [ ]:
queens_data=queens_data.reset_index(drop=True)

In [ ]:
queens_data.loc[0, 'Latitude'] 

In [ ]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )

In [ ]:
print(queens_venues.shape)
queens_venues.head()

In [ ]:
#Get venues for Staten Island
staten_island_data=df.loc[df['Borough']=='Staten Island'].copy()

In [ ]:
staten_island_data=staten_island_data.reset_index(drop=True)

In [ ]:
staten_island_data.loc[0, 'Latitude'] 

In [ ]:
staten_island_venues = getNearbyVenues(names=staten_island_data['Neighborhood'],
                                   latitudes=staten_island_data['Latitude'],
                                   longitudes=staten_island_data['Longitude']
                                  )

In [ ]:
print(staten_island_venues.shape)
staten_island_venues.head()

In [ ]:
#Create dataframe for all neighborhoods
foursquare_venues_new=pd.concat([manhattan_venues, brooklyn_venues, bronx_venues, queens_venues, staten_island_venues], axis=0, ignore_index=True)

In [ ]:
#Check 'tricky' neighborhoods just in case
foursquare_venues_new[foursquare_venues_new['Neighborhood'].str.contains('Chelsea')]

In [ ]:
#Write parsed data to the file to share with the team
foursquare_venues_new.to_csv('foursquare_venues_new.csv')